In [1]:
import logging
import os
import socket
import sys
import time

import numpy as np
import tenseal as ts

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, f1_score, recall_score, precision_score, accuracy_score
                           
os.chdir("/home/apignet/homomorphic-encryption/ckks_titanic/")
from src.features import build_features
from models import encrypted_LR
from models import unencrypted_LR

In [2]:
%load_ext memory_profiler

In [3]:
#fileHandler = logging.FileHandler("{0}/{1}.log".format(LOG_PATH, LOG_FILENAME))
streamHandler = logging.StreamHandler(sys.stdout)
logging.basicConfig(format="%(asctime)s  [%(levelname)-8.8s]  %(message)s", 
                    datefmt='%m/%d/%Y %I:%M:%S %p', 
                    level = logging.INFO, 
                    handlers=[ streamHandler]
                   )

In [2]:
HOST = '127.0.0.1'  # The server's hostname or IP address
PORT = 65431        # The port used by the server
PACKETS_SIZE = 16
SENTINEL = b'BREAK'


In [5]:
%%memit
logging.info('Definition of safety parameters...')
timer = time.time()
# context = ts.context(ts.SCHEME_TYPE.CKKS, 32768,
# coeff_mod_bit_sizes=[60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60])
#context = ts.context(ts.SCHEME_TYPE.CKKS, 8192, coeff_mod_bit_sizes=[40, 21, 21, 21, 21, 21, 21, 40])

context = ts.context(ts.SCHEME_TYPE.CKKS, 16384, coeff_mod_bit_sizes=[60, 40, 40, 40, 40, 40, 40,40, 60])
context.global_scale = pow(2, 40)
logging.info("Done. " + str(round(time.time() - timer, 2)) + " seconds")


logging.info('Generation of the secret key...')
timer = time.time()
secret_key = context.secret_key()
context.make_context_public() #drop the relin keys, the galois keys, and the secret keys. 
logging.info("Done. " + str(round(time.time() - timer, 2)) + " seconds")
logging.info('Generation of the Galois Key...')
timer = time.time()
context.generate_galois_keys(secret_key)
logging.info("Done. " + str(round(time.time() - timer, 2)) + " seconds")
logging.info('Generation of the Relin Key...')
timer = time.time()
context.generate_relin_keys(secret_key)
logging.info("Done. " + str(round(time.time() - timer, 2)) + " seconds")
if context.is_public():
    logging.info("The context is now public, the context do not hold the secret key anymore, and decrypt \
    methods need the secret key to be provide,")

logging.info("serialization of the context is starting...")
timer = time.time()
b_context = context.serialize()
logging.info("Serialization done in %s seconds" %(time.time()-timer))


08/05/2020 01:40:07 PM  [INFO    ]  Definition of safety parameters...
08/05/2020 01:40:07 PM  [INFO    ]  Done. 0.31 seconds
08/05/2020 01:40:07 PM  [INFO    ]  Generation of the secret key...
08/05/2020 01:40:07 PM  [INFO    ]  Done. 0.0 seconds
08/05/2020 01:40:07 PM  [INFO    ]  Generation of the Galois Key...
08/05/2020 01:40:10 PM  [INFO    ]  Done. 3.36 seconds
08/05/2020 01:40:10 PM  [INFO    ]  Generation of the Relin Key...
08/05/2020 01:40:10 PM  [INFO    ]  Done. 0.13 seconds
08/05/2020 01:40:10 PM  [INFO    ]  The context is now public, the context do not hold the secret key anymore, and decrypt     methods need the secret key to be provide,
08/05/2020 01:40:10 PM  [INFO    ]  serialization of the context is starting...
08/05/2020 01:41:00 PM  [INFO    ]  Serialization done in 49.51666307449341 seconds
peak memory: 1561.87 MiB, increment: 1427.10 MiB


In [3]:

alice = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
alice.connect((HOST, PORT))
print(alice.recv(1024))

ConnectionRefusedError: [Errno 111] Connection refused

In [7]:
data = b_context

In [8]:
for n in range(len(data) // PACKETS_SIZE + 1):
    alice.send(data[n * PACKETS_SIZE: (n + 1) * PACKETS_SIZE])
alice.send(SENTINEL)

5

In [11]:
alice.close()

In [ ]:
alice.sendall(b'Hello, pcihe')

In [ ]:
print(type(alice.recv))

print('Received', repr(pk.loads(data)))

In [9]:
data = b'KILLBOB'

In [ ]:
data == b'KILLBOB'

In [27]:
alice.send(b'KILLBOB')

7

In [28]:
alice.send(SENTINEL)

5

In [12]:
None in [None]

True